In [5]:
!pip install ultralytics

In [6]:
!unrar x frames_folder.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from frames_folder.rar

Creating    frames_folder                                             OK
Extracting  frames_folder/img_00001.jpg                                    0%  OK 
Extracting  frames_folder/img_00002.jpg                                    1%  OK 
Extracting  frames_folder/img_00003.jpg                                    2%  OK 
Extracting  frames_folder/img_00004.jpg                                    2%  OK 
Extracting  frames_folder/img_00005.jpg                                    3%  OK 
Extracting  frames_folder/img_00006.jpg                                    3%  OK 
Extracting  frames_folder/img_00007.jpg                                    4%  OK 
Extracting  frames_folder/img_00008.jpg                                    4%  OK 
Extracting  frames_folder/img_00009.jpg                                    4

In [7]:
!pip install deep_sort_realtime


In [10]:
import os
import cv2
import csv
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# Load the model
model = YOLO('yolov8m.pt')

# Initialize DeepSORT
deep_sort = DeepSort()

# Directory containing the raw frames
frames_dir = 'frames_folder'  # Update with the path to your folder of frames
frame_files = sorted([f for f in os.listdir(frames_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])

# Create a directory to save the annotated frames
output_dir = 'tracked_frames'
os.makedirs(output_dir, exist_ok=True)

# Prepare the CSV file to save tracking data
csv_file_path = 'tracking_data.csv'
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['frame', 'track_id', 'x1', 'y1', 'x2', 'y2'])  # Header

    # Class ID for cows
    cow_class_id = 19

    # Iterate over each frame file
    for frame_idx, frame_file in enumerate(frame_files):
        # Read the frame
        frame_path = os.path.join(frames_dir, frame_file)
        frame = cv2.imread(frame_path)

        # Perform detection on the frame
        results = model(frame)

        # Prepare detections for tracking
        detections = []
        for result in results:
            for detection in result.boxes:
                class_id = int(detection.cls.item())  # Get the class ID of the detection
                if class_id == cow_class_id:
                    # Get the bounding box coordinates and confidence score
                    x1, y1, x2, y2 = detection.xyxy[0].tolist()  # Convert to list
                    confidence = float(detection.conf.item())  # Get confidence score
                    # Append detection in the correct format for DeepSORT: [[x1, y1, x2, y2], confidence]
                    detections.append([[x1, y1, x2, y2], confidence])

        # Ensure detections are in the form [[x1, y1, x2, y2], confidence]
        if detections:
            # Update tracks with DeepSORT
            trackers = deep_sort.update_tracks(detections, frame=frame)
        else:
            trackers = []

        # Render the bounding boxes on the frame
        annotated_frame = frame.copy()  # Create a copy to annotate

        # Write tracking data to CSV file
        for track in trackers:
            x1, y1, x2, y2 = track.to_tlbr()  # Convert from tracker format to bounding box format
            track_id = track.track_id
            # Draw the bounding box on the frame
            cv2.rectangle(annotated_frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(annotated_frame, f'Track ID: {track_id}', (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Write the tracking data to the CSV
            csv_writer.writerow([frame_idx, track_id, x1, y1, x2, y2])

        # Save the annotated frame
        output_path = os.path.join(output_dir, f'frame_{frame_idx:04d}.jpg')
        cv2.imwrite(output_path, annotated_frame)

print(f"Frames saved to {output_dir}")
print(f"Tracking data saved to {csv_file_path}")



0: 384x640 1 person, 2 cars, 8 cows, 26.1ms
Speed: 2.3ms preprocess, 26.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 cars, 1 truck, 7 cows, 26.8ms
Speed: 3.1ms preprocess, 26.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 cars, 1 truck, 7 cows, 25.3ms
Speed: 3.0ms preprocess, 25.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 2 cars, 1 truck, 6 cows, 25.3ms
Speed: 3.1ms preprocess, 25.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 7 cows, 25.3ms
Speed: 3.7ms preprocess, 25.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 1 truck, 8 cows, 25.3ms
Speed: 3.6ms preprocess, 25.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 1 truck, 8 cows, 25.3ms
Speed: 3.6ms preprocess, 25.3ms inference, 1.2ms postpro